In [1]:
import numpy as np
import random
from random import shuffle


import os
import sys
sys.path.append( os.path.abspath("../../workspace/python-work/python 3/cs230_deeplearning_implement/ec_code/")  )
from class5.week1.dinosaurus_island.cllm_utils import *

#字符级语言模型 - 恐龙岛
#已经收集了所有恐龙名字，并编入了这个数据集,
# 为了构建字符级语言模型来生成新的名称，模型将学习不同的名称模式，并随机生成新的名字。

#构建模型中的模块，将来构建整个模型中的两个重要的模块：
# 梯度修剪：避免梯度爆炸
# 取样:一种用来产生字符的技术


In [2]:
data = open(r'C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week1\dinosaurus_island\dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
# 这些字符是a-z（26个英文字符）加上“\n”（换行字符），在这里换行字符起到
# 了在视频中类似的EOS（句子结尾）的作用，这里表示了名字的结束而不是句子的结尾。
# 下面将创建一个字典，每个字符映射到0-26的索引，然后再创建一个字典，
# 它将该字典将每个索引映射回相应的字符字符，它会帮助我们找出softmax层的概率分布输出中的字符。
# 我们来创建char_to_ix 与 ix_to_char字典。
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
# 梯度修剪：避免梯度爆炸
#整个循环结构通常包括前向传播、成本计算、反向传播和参数更新。
# 在更新参数之前，我们将在需要时执行梯度修剪，以确保我们的梯度不是“爆炸”的。
#梯度修剪：梯度向量的每一个元素都被限制在[−N，N] [-N，N][−N，N]的范围，
#       通俗的说，有一个maxValue（比如10），如果梯度的任何值大于10，那么它将被设置为10，
#       如果梯度的任何值小于-10，那么它将被设置为-10，如果它在-10与10之间，那么它将不变。
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. 
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    return gradients





# GRADED FUNCTION: sample   
# 采样  ==============>zero to many
def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN
    Arguments:
    parameters --dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix --dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.
    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    # 从parameters 中获取参数
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]   #by.shape =>  (n_y,m)  这个model中n_y=n_x
    n_a = Waa.shape[1]         #Waa.shape ==> (n_a,n_a)
    # 创建独热向量x
    x = np.zeros((vocab_size,1))
    # 使用0初始化a_prev
    a_prev = np.zeros((n_a,1))
    # 创建索引的空列表，这是包含要生成的字符的索引的列表。
    indices = []
    # IDX是检测换行符的标志，我们将其初始化为-1。
    idx = -1
    # 循环遍历时间步骤t。在每个时间步中，从概率分布中抽取一个字符，
    # 并将其索引附加到“indices”上，如果我们达到50个字符，
    #（我们应该不太可能有一个训练好的模型），我们将停止循环，这有助于调试并防止进入无限循环
    counter = 0
    newline_character = char_to_ix["\n"]
    
    while (idx != newline_character and counter < 50):
          # 计算隐藏单元的激活值：a⟨t⟩=tanh(W_aa * a⟨t−1⟩+W_ax * x⟨t⟩+ba)       (n_a,m)
          #2计算              yˆ⟨t⟩= softmax (W_ya* a⟨t⟩+by)                 (n_y,m)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        # 设定随机种子
        np.random.seed(counter + seed)
        # 从概率分布y中抽取词汇表中字符的索引
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
          #index 是int，是y中概率最大数的索引
        # 添加到索引中
        indices.append(idx)
        # 步骤4:将输入字符重写为与采样索引对应的字符。
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        # 更新a_prev为a
        a_prev = a 
        # 累加器
        seed += 1
        counter +=1
   #表示 名称的末尾
    if(counter == 50):
        indices.append(char_to_ix["\n"])
    return indices




#gd梯度下降的一次迭代
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    执行训练模型的单步优化。
    
    参数：
        X -- 整数列表，其中每个整数映射到词汇表中的字符。
        Y -- 整数列表，与X完全相同，但向左移动了一个索引。
        a_prev -- 上一个隐藏状态
        parameters -- 字典，包含了以下参数：
                        Wax -- 权重矩阵乘以输入，维度为(n_a, n_x)
                        Waa -- 权重矩阵乘以隐藏状态，维度为(n_a, n_a)
                        Wya -- 隐藏状态与输出相关的权重矩阵，维度为(n_y, n_a)
                        b -- 偏置，维度为(n_a, 1)
                        by -- 隐藏状态与输出相关的权重偏置，维度为(n_y, 1)
        learning_rate -- 模型学习的速率
    
    返回：
        loss -- 损失函数的值（交叉熵损失）
        gradients -- 字典，包含了以下参数：
                        dWax -- 输入到隐藏的权值的梯度，维度为(n_a, n_x)
                        dWaa -- 隐藏到隐藏的权值的梯度，维度为(n_a, n_a)
                        dWya -- 隐藏到输出的权值的梯度，维度为(n_y, n_a)
                        db -- 偏置的梯度，维度为(n_a, 1)
                        dby -- 输出偏置向量的梯度，维度为(n_y, 1)
        a[len(X)-1] -- 最后的隐藏状态，维度为(n_a, 1)
    """
    # 前向传播
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    # 反向传播
    gradients, a = rnn_backward(X, Y, parameters, cache)
    # 梯度修剪，[-5 , 5]
    gradients = clip(gradients,5)
    # 更新参数
    parameters =update_parameters(parameters,gradients,learning_rate)
    
    return loss, gradients, a[len(X)-1]



In [5]:
# GRADED FUNCTION: model
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, 
          dino_names = 7, vocab_size = 27):
    """
    训练模型并生成恐龙名字
    参数：
        data -- 语料库
        ix_to_char -- 索引映射字符字典
        char_to_ix -- 字符映射索引字典
        num_iterations -- 迭代次数
        n_a -- RNN单元数量
        dino_names -- 每次迭代中采样的数量
        vocab_size -- 在文本中的唯一字符的数量
    返回：
        parameters -- 学习后了的参数
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open(r"C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week1\dinosaurus_island\dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        # 定义一个训练样本
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        #每次训练一个样本=======================》随机梯度下降
        # Perform one optimization step:
        # Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        # 使用延迟来保持损失平滑,这是为了加速训练。
        loss = smooth(loss, curr_loss)
        
        # 每2000次迭代，通过sample()生成“\n”字符，检查模型是否学习正确
        if j % 2000 == 0:
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                # 采样 no to many  生成名字
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                #================important=====================
                #================important=====================
                #================important=====================
                #================important=====================
                #================important=====================
                # 为了得到相同的效果，随机种子+1
                seed += 1
            
            print("\n")
    return parameters

In [6]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.097224

Nkzxwtdmfqoeyhsqwasjjjvu
Kneb
Kzxwtdmfqoeyhsqwasjjjvu
Neb
Zxwtdmfqoeyhsqwasjjjvu
Eb
Xwtdmfqoeyhsqwasjjjvu




Iteration: 2000, Loss: 27.888091

Mivrosaurus
Inda
Ivtos
Macaisjeeansaurosopeeyalos
Xusedolopaurus
Ca
Tos




Iteration: 4000, Loss: 25.632304

Niussianesaurus
Inedaisilachusatorsaurus
Ivusceolosaurus
Necalpsaurus
Wusndonosaurus
Caaeson
Tosaurus




Iteration: 6000, Loss: 24.802620

Ngytosaurus
Inda
Kytosaurus
Nda
Xusmangosaurus
Ca
Tosaurus




Iteration: 8000, Loss: 24.252629

Onyusanichomurosaurus
Kida
Lyus
Ola
Xusagnchukuqosaurus
Ca
Trohichus




Iteration: 10000, Loss: 23.627444

Mivuselichojylosaurus
Klecalosaurus
Lyuskdleosaurus
Mecalosaurus
Xussangosaurus
Dabeosaurus
Toremaronykusaerasaurcisaurus




Iteration: 12000, Loss: 23.391712

Mixusteoosaurus
Inecalosaurus
Jytosaurus
Meeakosaurus
Xusteopoolus
Cabatron
Trohilosaurus


